In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#!pip install bs4

In [3]:
%aimport easysnowdata

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import tqdm
import contextily as ctx

import xarray as xr
import glob

#import logging
#logging.basicConfig(format='%(levelname)s:%(message)s',level=logging.INFO)

In [4]:
#!pip install --upgrade xee


In [9]:
!earthengine authenticate --quiet

Paste the following address into a web browser:

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=http%3A%2F%2Flocalhost%3A8085&response_type=code&code_challenge=W6DhvewkXdP9klnp2gJ8DkMzG2ED_4mQzOB0zVpRTN4&code_challenge_method=S256

On the web page, please authorize access to your Earth Engine account and copy the authentication code. Next authenticate with the following command:

    earthengine authenticate --code-verifier=uq2BXy4f13NAeSq2FIoMDaDPeAyG_z77Ea4cgTlFu8E --authorization-code=PLACE_AUTH_CODE_HERE

gio: https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww

In [5]:
import ee
import xarray

In [7]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')


In [15]:
huc02 = ee.FeatureCollection("USGS/WBD/2017/HUC02")

In [16]:
huc02

In [17]:
huc02_gdf = ee.data.computeFeatures({
    'expression': huc02,
    'fileFormat': 'GEOPANDAS_GEODATAFRAME'
})


In [1]:
#huc02_gdf.explore()

In [6]:
huc02_gdf = ee.data.listFeatures({
    'assetId': 'USGS/WBD/2017/HUC02',
    #'region': wa.geometry().getInfo(),
    #'filter': 'ORDER >= 3',
    'fileFormat': 'GEOPANDAS_GEODATAFRAME'
})

: 

In [ ]:
huc02_gdf

In [25]:
bbox_ee = ee.Geometry.Rectangle(153.0, -43.0, 154.0, -42.0)


In [26]:
ic = ee.ImageCollection('JAXA/ALOS/PALSAR-2/Level2_2/ScanSAR').filterDate('2020-10-05', '2021-03-31')
ds = xarray.open_dataset(ic, geometry=bbox_ee,engine='ee')


In [27]:
ds

<xarray.Dataset> Size: 188kB
Dimensions:  (time: 7834, lon: 1, lat: 1)
Coordinates:
  * time     (time) datetime64[ns] 63kB 2020-10-05T00:59:21.877000 ... 2021-0...
  * lon      (lon) float32 4B 153.5
  * lat      (lat) float32 4B -42.5
Data variables:
    HH       (time, lon, lat) float32 31kB ...
    HV       (time, lon, lat) float32 31kB ...
    LIN      (time, lon, lat) float32 31kB ...
    MSK      (time, lon, lat) float32 31kB ...
Attributes:
    crs:      EPSG:4326

In [28]:
ds['MSK'].isel(time=50).plot()

TypeError: No numeric data to plot.